## LECTURA DE FICHEROS

## INFO DE PROYECTO

### NOTAS

churn -> 30 DÍAS sin contrato


Clientes especiales: 
* TODO 0 -> YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=

## RUTAS DE CONFIGURACIÓN

In [5]:
RUTA_STORAGE = "kkboxschurn/"

In [2]:
def create_spark_session(app_name="Modelo de propension"):
    spark_session = SparkSession.builder \
        .appName(app_name) \
        .getOrCreate()
            
    spark_session.sparkContext.setLogLevel("WARN")
    spark.conf.set("spark.sql.crossJoin.enabled", "true")
       
    return spark_session

session = create_spark_session()
sc = session.sparkContext

NameError: name 'SparkSession' is not defined

## IMPORTACIÓN DE LIBRERÍAS

In [3]:
 #! pip install datalab\n"
import json
import pprint
import subprocess
import pyspark
import time
from pyspark.sql import SQLContext,DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import udf, array, col, explode, lit, struct, rank, desc,to_date, add_months, row_number, isnan, isnull, when, count
from pyspark.sql.types import IntegerType, DoubleType ,Row , StructType
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.linalg import Vector
from pyspark.mllib.stat import Statistics
from typing import Iterable 
import pandas as pd
import numpy as np
from functools import reduce  
#import datalab.storage as gcs\n",
from pyspark.storagelevel import StorageLevel 
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

ImportError: No module named 'pyspark'

## TRANSFORMACIÓN DE FICHEROS

### Esto es para pasar de csv a parquet (ficheros distribuidos)

In [4]:
transactions = spark.read.csv("gs://"+str(RUTA_STORAGE)+"input/transactions.csv", header=True)
transactions.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/transactions")

members_v3 = spark.read.csv("gs://"+str(RUTA_STORAGE)+"input/members_v3.csv", header=True)
members_v3.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/members_v3")

train_v2 = spark.read.csv("gs://"+str(RUTA_STORAGE)+"input/train_v2.csv", header=True)
train_v2.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/train_v2")

train = spark.read.csv("gs://"+str(RUTA_STORAGE)+"input/train.csv", header=True)
train.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/train")

transactions_v2 = spark.read.csv("gs://"+str(RUTA_STORAGE)+"input/transactions_v2.csv", header=True)
transactions_v2.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/transactions_v2")

user_logs = spark.read.csv("gs://"+str(RUTA_STORAGE)+"input/user_logs.csv", header=True)
user_logs.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/user_logs")

user_logs_v2 = spark.read.csv("gs://"+str(RUTA_STORAGE)+"input/user_logs_v2.csv", header=True)
user_logs_v2.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/user_logs_v2")
   

NameError: name 'spark' is not defined

### Leemos los ficheros almacenados en .parquet

In [4]:
transactions = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/transactions")
transactions.createOrReplaceTempView("transactions")

members_v3 = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/members_v3")
members_v3.createOrReplaceTempView("members_v3")

train_v2 = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/train_v2")
train_v2.createOrReplaceTempView("train_v2")

train = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/train")
train.createOrReplaceTempView("train")

transactions_v2 = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/transactions_v2")
transactions_v2.createOrReplaceTempView("transactions_v2")

user_logs_v2 = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/user_logs_v2")
user_logs_v2.createOrReplaceTempView("user_logs_v2")

user_logs = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/user_logs")
user_logs.createOrReplaceTempView("user_logs")

### Unimos los ficheros log y log2   y transacciones y transacciones2

In [25]:
total_elementos_log = sqlContext.sql("select msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs, \
                                             from_unixtime(unix_timestamp(date, 'yyyyMMdd')) as date_t \
                                      from user_logs \
                                      union all  \
                                      select msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs, \
                                      from_unixtime(unix_timestamp(date, 'yyyyMMdd')) as date_t \
                                      from user_logs_v2")

total_elementos_transactions = sqlContext.sql("select msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date, \
                       from_unixtime(unix_timestamp(transaction_date, 'yyyyMMdd')) as transaction_date_t, \
                       date_sub(from_unixtime(unix_timestamp(membership_expire_date, 'yyyyMMdd')),1) as membership_expire_date_t \
                from transactions \
                union all \
                select msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date, \
                       from_unixtime(unix_timestamp(transaction_date, 'yyyyMMdd')) as transaction_date_t, \
                       date_sub(from_unixtime(unix_timestamp(membership_expire_date, 'yyyyMMdd')),1) as membership_expire_date_t \
                from transactions_v2 \
                ")

total_elementos_log.cache()
total_elementos_transactions.cache()

total_elementos_log.createOrReplaceTempView("total_elementos_log")
total_elementos_transactions.createOrReplaceTempView("total_elementos_transactions")

### Relacionamos todos los elementos del log con cada transacción 
TODO:(Solucionar el tema de fechas solapadas)

In [65]:
pre_salida = sqlContext.sql("select t.msno as msno ,transaction_date,membership_expire_date,membership_expire_date_t , \
                        payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid, \
                        is_auto_renew,  \
                        date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs  \
                from total_elementos_transactions t \
                left join total_elementos_log l on \
                l.msno = t.msno and transaction_date_t <= date_t and  membership_expire_date_t >= date_t  ")


pre_salida.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/pre_salida")

### Eliminamos las transacciones que no tienen ninguna operación en el log

In [82]:
pre_salida = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/pre_salida")
pre_salida.createOrReplaceTempView("pre_salida")

pre_salida_filtrada =  sqlContext.sql("select msno,transaction_date,membership_expire_date, \
                 from_unixtime(unix_timestamp(membership_expire_date_t, 'yyyy-MM-dd')) as membership_expire_date_t, \
                 payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew, \
                 from_unixtime(unix_timestamp(date, 'yyyyMMdd')) as date, \
                 num_25,num_50,num_75,num_985,num_100,num_unq,total_secs \
                       from pre_salida where date is not null")
pre_salida_filtrada.cache()
pre_salida_filtrada.createOrReplaceTempView("pre_salida_filtrada")

### Creamos las variables a nivel de transacción

In [87]:
from pyspark.sql.functions  import date_format

salida = sqlContext.sql("select msno, transaction_date,membership_expire_date, \
                    count(*) as dias_contratados,\
                    sum(IF(total_secs >0 ,1,0)) as dias_conexion,\
                    sum(IF( num_25 >0 ,1,0)) as num_25,\
                    sum(IF( num_50 >0 ,1,0)) as num_50,\
                    sum(IF( num_75 >0 ,1,0)) as num_75,\
                    sum(IF( num_985 >0 ,1,0)) as num_985,\
                    sum(IF( num_100 >0 ,1,0)) as num_100,\
                    sum(IF( num_unq >0 ,1,0)) as num_unq,\
                    sum(IF(total_secs >0 ,1,0)) as total_secs,\
                    \
                    sum(IF(date = s.membership_expire_date_t and total_secs >0 ,1,0)) as dias_conexion_ld,\
                    sum(IF(date = s.membership_expire_date_t and num_25 >0 ,1,0)) as num_25_ld,\
                    sum(IF(date = s.membership_expire_date_t and num_50 >0 ,1,0)) as num_50_ld,\
                    sum(IF(date = s.membership_expire_date_t and num_75 >0 ,1,0)) as num_75_ld,\
                    sum(IF(date = s.membership_expire_date_t and num_985 >0 ,1,0)) as num_985_ld,\
                    sum(IF(date = s.membership_expire_date_t and num_100 >0 ,1,0)) as num_100_ld,\
                    sum(IF(date = s.membership_expire_date_t and num_unq >0 ,1,0)) as num_unq_ld,\
                    sum(IF(date = s.membership_expire_date_t and total_secs >0 ,1,0)) as total_secs_ld, \
                \
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and total_secs >0 ,1,0)) as dias_conexion_l7d,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_25 >0 ,1,0)) as num_25_l7d,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_50 >0 ,1,0)) as num_50_l7d,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_75 >0 ,1,0)) as num_75_l7d,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_985 >0 ,1,0)) as num_985_l7d,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_100 >0 ,1,0)) as num_100_l7d,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_unq >0 ,1,0)) as num_unq_l7d,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and total_secs >0 ,1,0)) as total_secs_l7d, \
                    \
                     \
                    sum(IF(date  >= date_sub(s.membership_expire_date_t,28) and total_secs >0 ,1,0)) as dias_conexion_l28d,\
                    sum(IF(date  >= date_sub(s.membership_expire_date_t,28) and num_25 >0 ,1,0)) as num_25_l28d,\
                    sum(IF(date  >= date_sub(s.membership_expire_date_t,28) and num_50 >0 ,1,0)) as num_50_l28d,\
                    sum(IF(date  >= date_sub(s.membership_expire_date_t,28) and num_75 >0 ,1,0)) as num_75_l28d,\
                    sum(IF(date  >= date_sub(s.membership_expire_date_t,28) and num_985 >0 ,1,0)) as num_985_l28d,\
                    sum(IF(date  >= date_sub(s.membership_expire_date_t,28) and num_100 >0 ,1,0)) as num_100_l28d,\
                    sum(IF(date  >= date_sub(s.membership_expire_date_t,28) and num_unq >0 ,1,0)) as num_unq_l28d,\
                    sum(IF(date  >= date_sub(s.membership_expire_date_t,28) and total_secs >0 ,1,0)) as total_secs_l28d, \
                    \
                    sum(num_25) as num_25v,\
                    sum(num_50) as num_50v,\
                    sum(num_75) as num_75v,\
                    sum(num_985) as num_985v,\
                    sum(num_100) as num_100v,\
                    sum(num_unq) as num_unqv,\
                    sum(total_secs) as total_secsv,\
                    \
                    sum(IF(date = s.membership_expire_date_t and num_25 >0 ,num_25,0)) as num_25_ldv,\
                    sum(IF(date = s.membership_expire_date_t and num_50 >0 ,num_50,0)) as num_50_ldv,\
                    sum(IF(date = s.membership_expire_date_t and num_75 >0 ,num_75,0)) as num_75_ldv,\
                    sum(IF(date = s.membership_expire_date_t and num_985 >0 ,num_985,0)) as num_985_ldv,\
                    sum(IF(date = s.membership_expire_date_t and num_100 >0 ,num_100,0)) as num_100_ldv,\
                    sum(IF(date = s.membership_expire_date_t and num_unq >0 ,num_unq,0)) as num_unq_ldv,\
                    sum(IF(date = s.membership_expire_date_t and total_secs >0 ,total_secs,0)) as total_secs_ldv, \
                \
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_25 >0 ,num_25,0)) as num_25_l7dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_50 >0 ,num_50,0)) as num_50_l7dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_75 >0 ,num_75,0)) as num_75_l7dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_985 >0 ,num_985,0)) as num_985_l7dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_100 >0 ,num_100,0)) as num_100_l7dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and num_unq >0 ,num_unq,0)) as num_unq_l7dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,7) and total_secs >0 ,total_secs,0)) as total_secs_l7dv, \
                    \
                     \
                    sum(IF(date >= date_sub(s.membership_expire_date_t,28) and num_25 >0 ,num_25,0)) as num_25_l28dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,28) and num_50 >0 ,num_50,0)) as num_50_l28dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,28) and num_75 >0 ,num_75,0)) as num_75_l28dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,28) and num_985 >0 ,num_985,0)) as num_985_l28dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,28) and num_100 >0 ,num_100,0)) as num_100_l28dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,28) and num_unq >0 ,num_unq,0)) as num_unq_l28dv,\
                    sum(IF(date >= date_sub(s.membership_expire_date_t,28) and total_secs >0 ,total_secs,0)) as total_secs_l28dv, \
                    \
                     max(IF(date <= s.membership_expire_date and total_secs >0 ,date,0)) as ultimos_dias_sin_conectarte\
                \
                from pre_salida_filtrada s \
               group by msno , transaction_date,membership_expire_date \
               ")

salida.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/elementos_modelo4")

### Creamos las variables a nivel de usuario

In [151]:
elementos_modelo4 = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/elementos_modelo4")
elementos_modelo4.createOrReplaceTempView("elementos_modelo4")

em = sqlContext.sql("select city,bd,gender,registered_via,registration_init_time,em.*, \
                     dense_rank() OVER (PARTITION BY em.msno ORDER BY transaction_date ) as num_suscr, \
                     datediff(from_unixtime(unix_timestamp(lead(transaction_date,1) OVER (PARTITION BY em.msno ORDER BY transaction_date  ), 'yyyyMMdd')),from_unixtime(unix_timestamp(membership_expire_date, 'yyyyMMdd'))) as sig_sub, \
                     sum(dias_contratados) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as dias_contratados_acumulado,\
                     sum(dias_conexion) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as dias_conexion_acumulado,\
                     sum(num_25) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_25_acumulado,\
                     sum(num_50) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_50_acumulado,\
                     sum(num_75) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_75_acumulado,\
                     sum(num_985) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_985_acumulado,\
                     sum(num_100) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_100_acumulado,\
                     sum(num_unq) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_unq_acumulado,\
                     sum(total_secs) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as total_secs_acumulado,\
                     sum(dias_conexion_ld) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as dias_conexion_ld_acumulado,\
                     sum(num_25_ld) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_25_ld_acumulado,\
                     sum(num_50_ld) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_50_ld_acumulado,\
                     sum(num_75_ld) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_75_ld_acumulado,\
                     sum(num_985_ld) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_985_ld_acumulado,\
                     sum(num_100_ld) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_100_ld_acumulado,\
                     sum(num_unq_ld) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_unq_ld_acumulado,\
                     sum(total_secs_ld) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as total_secs_ld_acumulado,\
                     sum(dias_conexion_l7d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as dias_conexion_l7d_acumulado,\
                     sum(num_25_l7d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_25_l7d_acumulado,\
                     sum(num_50_l7d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_50_l7d_acumulado,\
                     sum(num_75_l7d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_75_l7d_acumulado,\
                     sum(num_985_l7d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_985_l7d_acumulado,\
                     sum(num_100_l7d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_100_l7d_acumulado,\
                     sum(num_unq_l7d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_unq_l7d_acumulado,\
                     sum(total_secs_l7d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as total_secs_l7d_acumulado,\
                     sum(dias_conexion_l28d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as dias_conexion_l28d_acumulado,\
                     sum(num_25_l28d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_25_l28d_acumulado,\
                     sum(num_50_l28d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_50_l28d_acumulado,\
                     sum(num_75_l28d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_75_l28d_acumulado,\
                     sum(num_985_l28d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_985_l28d_acumulado,\
                     sum(num_100_l28d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_100_l28d_acumulado,\
                     sum(num_unq_l28d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_unq_l28d_acumulado,\
                     sum(total_secs_l28d) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as total_secs_l28d_acumulado,\
                     sum(num_25v) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_25v_acumulado,\
                     sum(num_50v) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_50v_acumulado,\
                     sum(num_75v) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_75v_acumulado,\
                     sum(num_985v) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_985v_acumulado,\
                     sum(num_100v) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_100v_acumulado,\
                     sum(num_unqv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_unqv_acumulado,\
                     sum(total_secsv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as total_secsv_acumulado,\
                     sum(num_25_ldv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_25_ldv_acumulado,\
                     sum(num_50_ldv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_50_ldv_acumulado,\
                     sum(num_75_ldv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_75_ldv_acumulado,\
                     sum(num_985_ldv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_985_ldv_acumulado,\
                     sum(num_100_ldv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_100_ldv_acumulado,\
                     sum(num_unq_ldv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_unq_ldv_acumulado,\
                     sum(total_secs_ldv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as total_secs_ldv_acumulado,\
                     sum(num_25_l7dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_25_l7dv_acumulado,\
                     sum(num_50_l7dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_50_l7dv_acumulado,\
                     sum(num_75_l7dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_75_l7dv_acumulado,\
                     sum(num_985_l7dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_985_l7dv_acumulado,\
                     sum(num_100_l7dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_100_l7dv_acumulado,\
                     sum(num_unq_l7dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_unq_l7dv_acumulado,\
                     sum(total_secs_l7dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as total_secs_l7dv_acumulado,\
                     sum(num_25_l28dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_25_l28dv_acumulado,\
                     sum(num_50_l28dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_50_l28dv_acumulado,\
                     sum(num_75_l28dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_75_l28dv_acumulado,\
                     sum(num_985_l28dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_985_l28dv_acumulado,\
                     sum(num_100_l28dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_100_l28dv_acumulado,\
                     sum(num_unq_l28dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as num_unq_l28dv_acumulado,\
                     sum(total_secs_l28dv) over (PARTITION BY em.msno order by transaction_date rows unbounded preceding) as total_secs_l28dv_acumulado \
                     from elementos_modelo4 em \
                     left join members_v3 me on me.msno = em.msno \
                     order by em.msno, transaction_date desc")

em.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/final_elementos_para_modelo2")

### Filtramos los elementos que no queremos en el modelo

In [ ]:
final_elementos_para_modelo = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/final_elementos_para_modelo2")
final_elementos_para_modelo.createOrReplaceTempView("final_elementos_para_modelo")
final_elementos_para_modelo.cache()

### Primera prueba_ filtramos los que no estén en el fichero de members y no sean la última transacción

In [175]:
sqlContext.sql("select gender, count(*) from final_elementos_para_modelo group by gender").show()

+------+--------+
|gender|count(1)|
+------+--------+
|  null| 8011185|
|female| 4505945|
|  male| 4964546|
+------+--------+



In [186]:
seleccionado_para_modelo = sqlContext.sql("select * from final_elementos_para_modelo where (sig_sub is not null or (sig_sub is null and membership_expire_date < '20170401')) and gender is not null")
seleccionado_para_modelo.cache()
seleccionado_para_modelo.createOrReplaceTempView("seleccionado_para_modelo")
#df.withColumn('norm_val', (df.val-min)/(max-min))

In [291]:
datos_preparados = sqlContext.sql("select  if (sig_sub <30,0,1) as label, array(dias_conexion/dias_contratados,if(gender == 'male',1,0)) as features  from seleccionado_para_modelo")

msno
transaction_date
-

## BORRAR

In [26]:
fecha_registro = sqlContext.sql("select msno ,min(transaction_date) as fecha_registro  from transactions group by msno")
fecha_registro.cache()
fecha_registro.createOrReplaceTempView("fecha_registro")

In [32]:
total_dias = sqlContext.sql("select date from user_logs  group by date order by date")
total_dias.cache()
total_dias.createOrReplaceTempView("total_dias")

In [57]:
elementos = sqlContext.sql("select td.date, fr.msno, \
                            IF(num_25 is NULL,0,num_25) as num_25,\
                            IF(num_50 is NULL,0,num_50) as num_50,\
                            IF(num_75 is NULL,0,num_75) as num_75,\
                            IF(num_985 is NULL,0,num_985) as num_985,\
                            IF(num_100 is NULL,0,num_100) as num_100,\
                            IF(num_unq is NULL,0,num_unq) as num_unq,\
                            IF(total_secs is NULL,0,total_secs) as total_secs\
                from total_dias td \
                join fecha_registro fr on fr.fecha_registro <= td.date\
                left join user_logs l on  l.date == td.date and l.msno == fr.msno \
                \
                ")
elementos.createOrReplaceTempView("elementos")
elementos.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/elementos")

In [210]:
susc_filtrada =  sqlContext.sql("select msno,transaction_date,membership_expire_date, \
                        dense_rank() OVER (PARTITION BY msno ORDER BY transaction_date ) as num_suscr, \
                        datediff(from_unixtime(unix_timestamp(lead(transaction_date,1) OVER (PARTITION BY msno ORDER BY transaction_date  ), 'yyyyMMdd')),from_unixtime(unix_timestamp(membership_expire_date, 'yyyyMMdd'))) as sig_sub, \
                        date_sub(from_unixtime(unix_timestamp(transaction_date, 'yyyyMMdd')),0) as transaction_date_t, \
                        date_sub(from_unixtime(unix_timestamp(membership_expire_date, 'yyyyMMdd')),1) as membership_expire_date_t \
                from transactions t  where msno = 'DwOmWms8HZUzNPPfiRwOMHj480D+GGqkDhNnSmZJm3s=' order by msno, transaction_date")

susc_total =  sqlContext.sql("select msno,transaction_date,membership_expire_date, \
                        dense_rank() OVER (PARTITION BY msno ORDER BY transaction_date ) as num_suscr, \
                        datediff(from_unixtime(unix_timestamp(lead(transaction_date,1) OVER (PARTITION BY msno ORDER BY transaction_date  ), 'yyyyMMdd')),from_unixtime(unix_timestamp(membership_expire_date, 'yyyyMMdd'))) as sig_sub, \
                        date_sub(from_unixtime(unix_timestamp(transaction_date, 'yyyyMMdd')),0) as transaction_date_t, \
                        date_sub(from_unixtime(unix_timestamp(membership_expire_date, 'yyyyMMdd')),1) as membership_expire_date_t \
                from transactions t   order by msno, transaction_date")

susc_filtrada.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/susc_filtrada")
susc_total.write.format("parquet").save("gs://"+str(RUTA_STORAGE)+"input_tratado/susc_total")

In [61]:
elementos = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/elementos")
elementos.createOrReplaceTempView("elementos")
susc_filtrada = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/susc_filtrada")
susc_filtrada.createOrReplaceTempView("susc_filtrada")
susc_total = sqlContext.read.parquet("gs://"+str(RUTA_STORAGE)+"input_tratado/susc_total")
susc_total.createOrReplaceTempView("susc_total")

In [238]:
susc_total_usada = sqlContext.sql("select msno,transaction_date,membership_expire_date,num_suscr,sig_sub, \
                       from_unixtime(unix_timestamp(transaction_date_t, 'yyyy-MM-dd')) as transaction_date_t, \
                       from_unixtime(unix_timestamp(membership_expire_date_t, 'yyyy-MM-dd')) as membership_expire_date_t   \
               from susc_total")
susc_total_usada.cache()
susc_total_usada.createOrReplaceTempView("susc_total_usada")
sqlContext.sql("select * from  susc_total_usada").count()

21547746

In [226]:
elemento_filtrado_actual = sqlContext.sql("select *, from_unixtime(unix_timestamp(date, 'yyyyMMdd')) as fecha_cast from elementos where msno = 'DwOmWms8HZUzNPPfiRwOMHj480D+GGqkDhNnSmZJm3s='  order by date desc")
elemento_filtrado_actual.cache()
elemento_filtrado_actual.createOrReplaceTempView("elemento_filtrado_actual")
sqlContext.sql("select * from  elemento_filtrado_actual").count()

446

In [241]:
elemento_filtrado_actual = sqlContext.sql("select *, from_unixtime(unix_timestamp(date, 'yyyyMMdd')) as fecha_cast from elementos order by date desc")
elemento_filtrado_actual.cache()
elemento_filtrado_actual.createOrReplaceTempView("elemento_filtrado_actual")
#sqlContext.sql("select * from  elemento_filtrado_actual").count()

## MODELO

### Primer modelo SVMWithSGD

In [298]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD
import array

parsedData = datos_preparados.rdd.map(lambda row: LabeledPoint(row.label, row.features))
model = SVMWithSGD.train(parsedData, iterations=100)
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.10354648437743193


### Segundo modelo Regresión logística

In [300]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

parsedData = datos_preparados.rdd.map(lambda row: LabeledPoint(row.label, row.features))
model = LogisticRegressionWithLBFGS.train(parsedData)
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.10354704024575606


### Tercer modelo Regresión logística (Linear least squares, Lasso, and ridge regression)

In [311]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from numpy import array
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.feature import StandardScalerModel


parsedData = datos_preparados.rdd.map(lambda row: LabeledPoint(row.label, row.features))
model = LinearRegressionWithSGD.train(parsedData, iterations=100, step=0.00000001)
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))


/usr/lib/spark/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


Training Error = 0.9999998888263352


### Cuarto modelo Bayes

In [309]:
#from pyspark.ml.classification import NaiveBayes
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#
#nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
#
#parsedData = datos_preparados.rdd.map(lambda row: LabeledPoint(row.label, row.features))
#
#model = nb.fit(parsedData)
#
#predictions = model.transform(parsedData)
#
#predictions.show()